# TACo staking analysis

## Get the authorizations from subgraph

In [1]:
import os
import pandas as pd
import dotenv
from web3 import Web3
from web3.middleware import ExtraDataToPOAMiddleware
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

dotenv.load_dotenv(".env")

deauth_period = 15724800
w3 = Web3(Web3.HTTPProvider(os.getenv('ETH_RPC_URL')))
taco_application_abi = '[ { "inputs": [ { "internalType": "address", "name": "", "type": "address" } ], "name": "stakingProviderInfo", "outputs": [ { "internalType": "address", "name": "operator", "type": "address" }, { "internalType": "bool", "name": "operatorConfirmed", "type": "bool" }, { "internalType": "uint64", "name": "operatorStartTimestamp", "type": "uint64" }, { "internalType": "uint96", "name": "authorized", "type": "uint96" }, { "internalType": "uint96", "name": "deauthorizing", "type": "uint96" }, { "internalType": "uint64", "name": "endDeauthorization", "type": "uint64" }, { "internalType": "uint96", "name": "tReward", "type": "uint96" }, { "internalType": "uint160", "name": "rewardPerTokenPaid", "type": "uint160" }, { "internalType": "uint64", "name": "endCommitment", "type": "uint64" }, { "internalType": "uint256", "name": "stub", "type": "uint256" }, { "internalType": "uint192", "name": "penaltyPercent", "type": "uint192" }, { "internalType": "uint64", "name": "endPenalty", "type": "uint64" } ], "stateMutability": "view", "type": "function" } ]'
taco_application = w3.eth.contract(
    address='0x347CC7ede7e5517bD47D20620B2CF1b406edcF07', abi=taco_application_abi)

transport = AIOHTTPTransport(
    url="https://subgraph.satsuma-prod.com/276a55924ce0/nucypher--102994/mainnet/api")
client = Client(transport=transport, fetch_schema_from_transport=True)

app_auths_query = gql(
    """
    query appAuthorizations {
        appAuthorizations(first: 1000) {
            amount
            amountDeauthorizing
            stake {
            id
            }
            appName
        }
    }
"""
)

auths_json = await client.execute_async(app_auths_query)
auths_json = auths_json['appAuthorizations']

for auth in auths_json:
    auth['stakingProvider'] = auth['stake']['id']
    del auth['stake']

auths = pd.DataFrame(auths_json)
auths['amount'] = auths['amount'].apply(int)
auths['amount'] = auths['amount'].div(10**18)
auths['amount'] = auths['amount'].apply(round)
auths['amountDeauthorizing'] = auths['amountDeauthorizing'].apply(int)
auths['amountDeauthorizing'] = auths['amountDeauthorizing'].div(10**18)
auths['amountDeauthorizing'] = auths['amountDeauthorizing'].apply(round)

taco_auths = auths[auths['appName'] == 'TACo']
taco_auths.set_index('stakingProvider', inplace=True)
taco_auths = taco_auths.drop(columns=['appName'])
taco_auths['relativeAmount'] = taco_auths['amount'] / \
    taco_auths['amount'].sum()
taco_auths['relativeAmount'] = taco_auths['relativeAmount'].apply(
    round, args=(4, ))
taco_auths.sort_values(by='amount', ascending=False, inplace=True)

## Get the deauthorizing end date from TACo app contract

Get the deauthorizing end date for stakes that are deauthorizing and the remaining staked amount is less than 40 000T

In [2]:
taco_auths['endDeauth'] = pd.NaT
for index, row in taco_auths.iterrows():
    remaining_amount = row['amount'] - row['amountDeauthorizing']
    if row['amountDeauthorizing'] > 0 and remaining_amount < 40000:
        st_prov_info = taco_application.functions.stakingProviderInfo(
            w3.to_checksum_address(index)).call()
        taco_auths.loc[index, 'endDeauth'] = pd.to_datetime(
            st_prov_info[5], unit='s')

## Get the TACo commitments fom subgraph

In [3]:
def commitment_status(endCommitment):
    if pd.isna(endCommitment):
        return 'No'
    elif pd.Timestamp.now() > endCommitment:
        return 'Expired'
    else:
        return 'Active'


commitments_query = gql(
    """
    query commitments {
        tacoCommitments(first: 1000) {
            id
            endCommitment
        }
    }
"""
)

commitments_json = await client.execute_async(commitments_query)
commitments_json = commitments_json['tacoCommitments']


commitments = pd.DataFrame(commitments_json)
commitments.rename(columns={'id': 'stakingProvider'}, inplace=True)
commitments.set_index('stakingProvider', inplace=True)
commitments['endCommitment'] = pd.to_datetime(
    commitments['endCommitment'].astype(int), unit='s')

taco_auths = taco_auths.join(commitments, how='left')
taco_auths['commitment'] = taco_auths['endCommitment'].apply(commitment_status)

## Calculate the minimum dauthorization date

Note that `endDeauth` is the date in which the TACo stake will be disabled (staked amount < 40000).

In [4]:
for index, row in taco_auths.iterrows():
    endDeauth = row['endDeauth']
    endCommitment = row['endCommitment']
    canDeauthorize = pd.isna(
        endCommitment) or endCommitment < pd.Timestamp.now()

    # if there is an active commitment
    if not canDeauthorize:
        # it has to wait until the commitment ends and then wait the deauthorization period (6 months)
        minDeauthDate = endCommitment + \
            pd.Timedelta(deauth_period, unit='seconds')
    # if there is no active commitment, and they are not deauthorizing by the moment
    elif pd.isna(endDeauth):
        # it has to wait the deauthorization period (6 months)
        minDeauthDate = pd.Timestamp.now().round(
            freq='s') + pd.Timedelta(deauth_period, unit='seconds')
    # if there is no active commitment, and they are already deauthorizing
    else:
        # it has to wait the remaining deauthorization time (less than 6 months)
        minDeauthDate = endDeauth

    taco_auths.loc[index, 'minDeauthDate'] = minDeauthDate

## Check if the stakes authorize tBTC as well

**TODO: to be finished**

In [5]:
# TODO: to be finished

tbtc_auths = auths[(auths['appName'] == 'tBTC') & (auths['amount'] > 0)]
tbtc_auths.set_index('stakingProvider', inplace=True)
tbtc_auths = tbtc_auths.drop(columns=['appName', 'amountDeauthorizing'])

rb_auths = auths[(auths['appName'] == 'Random Beacon') & (auths['amount'] > 0)]
rb_auths.set_index('stakingProvider', inplace=True)
rb_auths = rb_auths.drop(columns=['appName', 'amountDeauthorizing'])

tbtc_rb_auths = tbtc_auths.join(
    rb_auths, how="right", lsuffix='_tbtc', rsuffix='_rb')
# tbtc_rb_auths.fillna(0, inplace=True)


## Get the relevant rituals participants from Coordinator contract

In [6]:
coordinator_abi = '[{ "inputs": [ { "internalType": "uint32", "name": "ritualId", "type": "uint32" } ], "name": "getParticipants", "outputs": [ { "components": [ { "internalType": "address", "name": "provider", "type": "address" }, { "internalType": "bool", "name": "aggregated", "type": "bool" }, { "internalType": "bytes", "name": "transcript", "type": "bytes" }, { "internalType": "bytes", "name": "decryptionRequestStaticKey", "type": "bytes" } ], "internalType": "struct Coordinator.Participant[]", "name": "", "type": "tuple[]" } ], "stateMutability": "view", "type": "function" }]'

w3 = Web3(Web3.HTTPProvider(os.getenv('POLYGON_URL_ENDPOINT')))
w3.middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)

coordinator = w3.eth.contract(address="0xE74259e3dafe30bAA8700238e324b47aC98FE755", abi=coordinator_abi)

def get_participants(ritual_data):
    participants = []
    for participant in ritual_data:
        participants.append(participant[0].lower())
    return participants

ritual34 = coordinator.functions.getParticipants(34).call()
ritual40 = coordinator.functions.getParticipants(40).call()

ritual34_participants = get_participants(ritual34)
ritual40_participants = get_participants(ritual40)

taco_auths['ritual34?'] = taco_auths.index.isin(ritual34_participants)
taco_auths['ritual40?'] = taco_auths.index.isin(ritual40_participants)

## Export to CSV

In [7]:
taco = pd.DataFrame(taco_auths, index=taco_auths.index, columns=[
                    'amount', 'relativeAmount', 'minDeauthDate', 'commitment', 'ritual34?', 'ritual40?'])
taco.to_csv('taco.csv', )